# Artificial Faces Recognition Project
----------
Importing libraries:

In [1]:
import os
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.constraints import max_norm
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Sequential
from sklearn.metrics import confusion_matrix
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from PIL import Image
import io
import glob
import random
import shutil


Setting up our training, validation and testing data folders:

In [2]:
train_frac = 0.6
validation_frac = 0.2

reset_dataset = False

#Let's check if the folders already exist, and if not, create them. We may want to create them again with different settings too
if not os.path.isdir('real_and_fake_face/train/fake') or reset_dataset:

    #Deleting pre existing folders if they exist
    if os.path.isdir('real_and_fake_face/train/fake'):
        shutil.rmtree('real_and_fake_face/train/real')
        shutil.rmtree('real_and_fake_face/train/fake')
        shutil.rmtree('real_and_fake_face/validation/real')    
        shutil.rmtree('real_and_fake_face/validation/fake')
        shutil.rmtree('real_and_fake_face/test/real')
        shutil.rmtree('real_and_fake_face/test/fake')

    #Creating empty folders
    os.makedirs('real_and_fake_face/train/real')
    os.makedirs('real_and_fake_face/train/fake')
    os.makedirs('real_and_fake_face/validation/real')    
    os.makedirs('real_and_fake_face/validation/fake')
    os.makedirs('real_and_fake_face/test/real')
    os.makedirs('real_and_fake_face/test/fake')

    #Separating the training, validation and test data
    real_files = os.listdir('real_and_fake_face/real_complete')
    random.shuffle(real_files)
    count_real = len(real_files)
    fake_files = os.listdir('real_and_fake_face/fake_complete')
    random.shuffle(fake_files)
    count_fake = len(fake_files)

    train_real = real_files[0:int(count_real*train_frac)]
    validation_real = real_files[int(count_real*train_frac):int(count_real*train_frac)+int(count_real*validation_frac)]
    test_real = real_files[int(count_real*train_frac)+int(count_real*validation_frac)::]

    train_fake = fake_files[0:int(count_fake*train_frac)]
    validation_fake = fake_files[int(count_fake*train_frac):int(count_fake*train_frac)+int(count_fake*validation_frac)]
    test_fake = fake_files[int(count_fake*train_frac)+int(count_fake*validation_frac)::]

    #Filling the empty folders
    for file in train_real:
        src = 'real_and_fake_face/real_complete/'+file
        dst = 'real_and_fake_face/train/real/'+file
        shutil.copyfile(src,dst)
    for file in validation_real:
        src = 'real_and_fake_face/real_complete/'+file
        dst = 'real_and_fake_face/validation/real/'+file
        shutil.copyfile(src,dst)
    for file in test_real:
        src = 'real_and_fake_face/real_complete/'+file
        dst = 'real_and_fake_face/test/real/'+file
        shutil.copyfile(src,dst)
    for file in train_fake:
        src = 'real_and_fake_face/fake_complete/'+file
        dst = 'real_and_fake_face/train/fake/'+file
        shutil.copyfile(src,dst)
    for file in validation_fake:
        src = 'real_and_fake_face/fake_complete/'+file
        dst = 'real_and_fake_face/validation/fake/'+file
        shutil.copyfile(src,dst)
    for file in test_fake:
        src = 'real_and_fake_face/fake_complete/'+file
        dst = 'real_and_fake_face/test/fake/'+file
        shutil.copyfile(src,dst)


Configuring the reading and preprocessing of data:

In [3]:
train_path = 'real_and_fake_face/train'
validation_path = 'real_and_fake_face/validation'
test_path = 'real_and_fake_face/test'

train_batches = ImageDataGenerator(preprocessing_function=tf.keras.applications.vgg16.preprocess_input) \
    .flow_from_directory(directory=train_path,target_size=(224,224),classes=['real','fake'],batch_size=10)
validation_batches = ImageDataGenerator(preprocessing_function=tf.keras.applications.vgg16.preprocess_input) \
    .flow_from_directory(directory=validation_path,target_size=(224,224),classes=['real','fake'],batch_size=10)
test_batches = ImageDataGenerator(preprocessing_function=tf.keras.applications.vgg16.preprocess_input) \
    .flow_from_directory(directory=test_path,target_size=(224,224),classes=['real','fake'],batch_size=10)

Found 1224 images belonging to 2 classes.
Found 408 images belonging to 2 classes.
Found 409 images belonging to 2 classes.


In [6]:
model = Sequential()
model.add(keras.layers.Conv2D(filters=32, kernel_size=(3,3),activation='relu',padding='same',input_shape=(224,224,3),\
    kernel_constraint=max_norm(0.5), bias_constraint=max_norm(0.5)))           
model.add(keras.layers.MaxPool2D(pool_size=(2,2),strides=2))
model.add(keras.layers.Conv2D(filters=64,kernel_size=(3,3),activation='relu',padding='same',\
    kernel_constraint=max_norm(0.5), bias_constraint=max_norm(0.5)))
model.add(keras.layers.MaxPool2D(pool_size=(2,2),strides=2))
model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(units=2,activation='softmax'))
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_2 (Conv2D)            (None, 224, 224, 32)      896       
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 112, 112, 32)      0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 112, 112, 64)      18496     
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 56, 56, 64)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 200704)            0         
_________________________________________________________________
dense_1 (Dense)              (None, 2)                 401410    
Total params: 420,802
Trainable params: 420,802
Non-trainable params: 0
________________________________________________

In [7]:
model.compile(optimizer=Adam(learning_rate=0.0001),loss='categorical_crossentropy',metrics=['accuracy'])
model.fit(x=train_batches,validation_data=validation_batches,epochs=10,verbose=2)

Train for 123 steps, validate for 41 steps
Epoch 1/10
123/123 - 77s - loss: 16.0581 - accuracy: 0.5400 - val_loss: 6.4154 - val_accuracy: 0.5613
Epoch 2/10
123/123 - 83s - loss: 3.4912 - accuracy: 0.6724 - val_loss: 3.4597 - val_accuracy: 0.5613
Epoch 3/10
123/123 - 87s - loss: 1.0649 - accuracy: 0.8211 - val_loss: 2.5908 - val_accuracy: 0.5931
Epoch 4/10
123/123 - 82s - loss: 0.3103 - accuracy: 0.9158 - val_loss: 2.3518 - val_accuracy: 0.5711
Epoch 5/10
123/123 - 85s - loss: 0.0925 - accuracy: 0.9673 - val_loss: 2.1708 - val_accuracy: 0.5882
Epoch 6/10
123/123 - 90s - loss: 0.0148 - accuracy: 0.9951 - val_loss: 2.1677 - val_accuracy: 0.5882
Epoch 7/10
123/123 - 83s - loss: 0.0031 - accuracy: 1.0000 - val_loss: 2.1891 - val_accuracy: 0.5956
Epoch 8/10
123/123 - 83s - loss: 0.0020 - accuracy: 1.0000 - val_loss: 2.2005 - val_accuracy: 0.5907
Epoch 9/10
123/123 - 82s - loss: 0.0017 - accuracy: 1.0000 - val_loss: 2.2119 - val_accuracy: 0.5931
Epoch 10/10
123/123 - 85s - loss: 0.0014 - accu